# Deep Learning practice

### Carga de datos

Empiezo cargando los datos.

In [1]:
import numpy as np
import pandas as pd

was_loaded = np.load('was_loaded.npy')
data = pd.read_csv('airbnb-listings.csv',sep=';', decimal='.')
data.shape

(14001, 89)

Limpio los registros sin imágenes y me creo una variable para ir guardando los índices que voy a eliminar. Así los elimino todos de golpe y me aseguro que los índices que guardo en el csv al final van a servir para el array de imágenes.

In [2]:
total_idx_to_drop = []
total_idx_to_drop += data[was_loaded!=1].index.tolist()

In [3]:
data.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

Me quedo únicamente con descripciones y variables numéricas (elimino IDs y enlaces). También elimino las variables Monthly Price y Weekly Price porque forman parte de la variable objetivo (el precio) y con ellas estaría haciendo trampa. También me quito algunas variables que carecen de contenido.

In [4]:
columns=['ID','Listing Url','Scrape ID','Thumbnail Url','Medium Url','Picture Url','XL Picture Url',
         'Host ID','Host URL','Host Thumbnail Url','Host Picture Url','Monthly Price','Weekly Price',
         'Experiences Offered','Host Acceptance Rate','Jurisdiction Names','Has Availability','Geolocation']

data = data.drop(columns=columns)

También voy a filtrar para quedarme únicamente con los registros de Madrid, ya que solamente quiero trabajar con los datos de Madrid.

In [5]:
total_idx_to_drop += data[(data.City != 'Madrid') & (data.City != 'madrid')].index.tolist()

# Ahora también puedo borrar esta columna y las que hacen referencia al país, que ya no me aportan nada.
data = data.drop(columns=['City','State','Country Code','Country','Smart Location'])

Como la variable objetivo es el precio, no tendría sentido entrenar con datos cuyo precio es desconocido, así que descartaré de mi dataset todos aquellos apartamentos cuyo precio sea nan.

In [6]:
total_idx_to_drop += data[data["Price"].isnull()].index.tolist()

Me creo una función para limpiar un poco por encima los datos categóricos

In [7]:
def clean_data(data, categorical):
    
    columns_to_drop = []
    idx_to_drop = []
    
    for col in categorical:
        counts = data[col].value_counts().tolist()
        if counts[0] > sum(counts)*0.95 or counts[0] < sum(counts)*0.10 or sum(data[col].isnull()) > sum(counts)/2:
            print(f'Deleting column {col} with {len(counts)} distinct values and {sum(data[col].isnull())} nulls')
            columns_to_drop.append(col)
        else:
            vals = data[col].value_counts().keys().tolist()
            for val, count in zip(vals,counts):
                if count < 25:
                    idx_to_drop += [i for i in data[data[col]==val].index.tolist() if i not in idx_to_drop]
    
    return columns_to_drop, idx_to_drop

In [8]:
categorical = ['Last Scraped','Host Since','Host Name','Host Response Time','Host Location',
               'Calendar Updated','Calendar last Scraped','Host Neighbourhood','Neighbourhood',
               'Neighbourhood Cleansed','Neighbourhood Group Cleansed','Zipcode','Market','Property Type',
               'Room Type','Bed Type','License','Cancellation Policy','First Review','Last Review']

dropcolumns, dropindex = clean_data(data, categorical)
total_idx_to_drop += dropindex
clean_data = data.drop(index=total_idx_to_drop, columns=dropcolumns)

Deleting column Last Scraped with 2 distinct values and 0 nulls
Deleting column Host Since with 2065 distinct values and 3 nulls
Deleting column Host Name with 3053 distinct values and 3 nulls
Deleting column Host Neighbourhood with 130 distinct values and 3709 nulls
Deleting column Neighbourhood with 96 distinct values and 5005 nulls
Deleting column Market with 7 distinct values and 50 nulls
Deleting column Bed Type with 5 distinct values and 0 nulls
Deleting column License with 314 distinct values and 13668 nulls
Deleting column First Review with 1629 distinct values and 3007 nulls
Deleting column Last Review with 720 distinct values and 3008 nulls


In [9]:
clean_data.shape

(9216, 56)

Me separo los datos en train, validation y test

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(clean_data, test_size=0.2, shuffle=True)
train, val = train_test_split(train, test_size=0.1, shuffle=True)

# Me guardo los datos para asegurarme de no modificarlos accidentalmente.
train.to_csv('./train.csv', sep=';', decimal='.', index=True)
val.to_csv('./val.csv', sep=';', decimal='.', index=True)
test.to_csv('./test.csv', sep=';', decimal='.', index=True)